In [4]:
import re
from PyPDF2 import PdfFileReader, PdfFileWriter


In [9]:
pdf_path = 'C:/Users/Jana/OneDrive/Dokumente/Desktop/Uni/Master/Thesis/PaperDatenextraktion/IARJSET7.pdf'
#pdf_path = open(str(filePath)+'Data Mining Tools.pdf', 'rb')

doc = fitz.open(pdf_path)
#page = doc[2]
text = doc.get_text("text")
print(doc.metadata)

NameError: name 'fitz' is not defined

In [8]:
re.findall(r'(?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (?:\d{1,2}, )?\d{2,4}',reader)

TypeError: expected string or bytes-like object